In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import matplotlib.font_manager as fm

# ==========================================
# [설정] 한글 폰트 설정 (WSL/Linux 환경)
# ==========================================
# 나눔고딕이 설치되어 있다고 가정 (/usr/share/fonts/truetype/nanum/NanumGothic.ttf)
# 폰트가 없다면 기본 폰트로 실행되지만 한글이 깨질 수 있습니다.
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

# ==========================================
# [1] 데이터 로드 및 길이 분석
# ==========================================
DATA_DIR = "data/processed/sections"
files = glob.glob(os.path.join(DATA_DIR, "*.md"))

print(f"총 파일 개수: {len(files)}")

doc_lengths = []
all_text = ""

for file_path in files:
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        doc_lengths.append(len(content))
        all_text += content + " "

# 통계 출력
print(f"\n=== 텍스트 길이 통계 (글자 수) ===")
print(f"최소 길이: {min(doc_lengths)}")
print(f"최대 길이: {max(doc_lengths)}")
print(f"평균 길이: {sum(doc_lengths) / len(doc_lengths):.2f}")

# 히스토그램 그리기
plt.figure(figsize=(10, 5))
sns.histplot(doc_lengths, bins=30, kde=True)
plt.title("Distribution of Section Lengths (Characters)")
plt.xlabel("Length")
plt.ylabel("Count")
plt.grid(True, alpha=0.3)
plt.show() # 혹은 plt.savefig('eda_length_dist.png')

# ==========================================
# [2] 청크(Chunk) 시뮬레이션
# ==========================================
# 실제 RAG에서 사용할 청크 크기가 적절한지 미리 테스트해봅니다.
from langchain_text_splitters import RecursiveCharacterTextSplitter

CHUNK_SIZE = 500
CHUNK_OVERLAP = 50

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

# 전체 텍스트를 청킹해보며 분포 확인
sample_chunks = splitter.split_text(all_text[:100000]) # 너무 많으면 샘플링
chunk_lengths = [len(c) for c in sample_chunks]

plt.figure(figsize=(10, 5))
sns.histplot(chunk_lengths, bins=30, color='orange', kde=True)
plt.title(f"Chunk Length Distribution (Size={CHUNK_SIZE}, Overlap={CHUNK_OVERLAP})")
plt.xlabel("Chunk Length (Characters)")
plt.ylabel("Count")
plt.axvline(CHUNK_SIZE, color='r', linestyle='--', label='Target Size')
plt.legend()
plt.show()

# ==========================================
# [3] 키워드 빈도 분석 (간단 버전)
# ==========================================
# 공백 기준으로 단어 분리 (정교한 분석을 위해선 Konlpy 사용 권장)
words = all_text.split()
# 2글자 이상인 단어만 필터링
words = [w for w in words if len(w) >= 2]

counter = Counter(words)
top_20 = counter.most_common(20)

print("\n=== 상위 20개 빈출 단어 ===")
for word, count in top_20:
    print(f"{word}: {count}")

# 막대 그래프
top_words, top_counts = zip(*top_20)
plt.figure(figsize=(12, 6))
sns.barplot(x=list(top_counts), y=list(top_words))
plt.title("Top 20 Frequent Words")
plt.show()